# Layout

+ opening
    + contents
    + files used
    + packages used

+ Scraping
    + Images
        + OCR
    + raw text
    + html
    + PDFs
    + word doc etc.
+ spidering
    + wikipedia
    + APIs
        + REST
        + tumblr
+ reading files
    + encodings
    + unicode
+ filtering
+ data structures
    + pandas


# Week 1 - Intro

Intro stuff ...

For this notebook we will be using the following packages

In [1]:
import requests #http requests
import bs4 #called 'BeautifulSoup', a html parser
import re #for regexs
import pandas #DataFrames
import urllib.parse #For joining urls

We will also be working on the following files/urls

In [2]:
wikipedia_base_url = 'https://en.wikipedia.org'
wikipedia_content_analysis = 'https://en.wikipedia.org/wiki/Content_analysis'
content_analysis_save = 'wikipedia_content_analysis.html'

# Scraping

Before we can start analyzing content we need to obtain it. Sometimes it will be provided to us before hand, but often we will need to download it. As a starting example we will attempt to download the wikipedia page on content analysis. The page is located at [https://en.wikipedia.org/wiki/Content_analysis](https://en.wikipedia.org/wiki/Content_analysis) so lets start with that.

We can do this by making an HTTP GET request to that url, a GET request is simply a request to the server to provide the contents given by some url. The other request we will be using in this class is called a POST request and requests the server to take some content we provide. While the Python standard library does have the ability do make GET requests we will be using the [_requests_](http://docs.python-requests.org/en/master/) package as it is _'the only Non-GMO HTTP library for Python'_, also it provides a nicer interface.

In [3]:
#wikipedia_content_analysis = 'https://en.wikipedia.org/wiki/Content_analysis'
requests.get('https://en.wikipedia.org/wiki/Content_analysis')

<Response [200]>

`'Response [200]'` means the server responded with what we asked for. If you get another number (e.g. 404) it likely means there was some kind of error, these codes are called HTTP response codes and a list of them can be found [here](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes). The response object contains all the data the server sent including the website's contents and the HTTP header. We are interested in the contents which we can access with the `.text` attribute.

In [4]:
wikiContentRequest = requests.get('https://en.wikipedia.org/wiki/Content_analysis')
print(wikiContentRequest.text[:1000])

<!DOCTYPE html>
<html class="client-nojs" lang="en" dir="ltr">
<head>
<meta charset="UTF-8"/>
<title>Content analysis - Wikipedia</title>
<script>document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );</script>
<script>(window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"Content_analysis","wgTitle":"Content analysis","wgCurRevisionId":735443188,"wgRevisionId":735443188,"wgArticleId":473317,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles needing cleanup from April 2008","All articles needing cleanup","Cleanup tagged articles without a reason field from April 2008","Wikipedia pages needing cleanup from April 2008","Articles needing expert attention with no reason or talk parameter","Articles needing expert attention from April 2008","All artic

This is not what we were looking for, because it is the start of the HTML that makes up the website. This is HTML and is meant to be read by computers. Luckily we have a computer to parse it for us. To do the parsing we will use [_Beautiful Soup_](https://www.crummy.com/software/BeautifulSoup/) which is a better parser than the one in the standard library.

In [5]:
wikiContentSoup = bs4.BeautifulSoup(wikiContentRequest.text, 'html.parser')
print(wikiContentSoup.text[:200])





Content analysis - Wikipedia
document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );
(window.RLQ=window.RLQ||[]).push(functio


This is better but there's a bunch of random whitespace and we have way more than just the text of the article. This is because what we requested is the whole webpage, not just the text for the article.

We need to extract only the text we care about, in order to do this we will need to inspect the html. One way to do this is to simply go to the website with a browser and use its inspection or view source tool, but if there is javascript or other dynamic loading occurring on the page it is very likely that what Python receives is not what you will see. So we will need to view what Python receives. To do this we can save the html `requests` obtained.

In [6]:
#content_analysis_save = 'wikipedia_content_analysis.html'

with open(content_analysis_save, mode='w', encoding='utf-8') as f:
    f.write(wikiContentRequest.text)

Now lets open the file (`wikipedia_content_analysis.html`) we just created with a web browser. It should look sort of like the original but missing all the images and formatting.

As there is very little standardization on structuring webpages figuring out how best to extract what you want is an art. Looking at this page it looks like all the main textual content is within `<p>`(paragraph) tags inside the `<body>` tag.

In [7]:
contentPTags = wikiContentSoup.body.findAll('p')
for pTag in contentPTags[:3]:
    print(pTag.text)

Content analysis is "a wide and heterogeneous set of manual or computer-assisted techniques for contextualized interpretations of documents produced by communication processes in the strict sense of that phrase (any kind of text, written, iconic, multimedia, etc.) or signification processes (traces and artifacts), having as ultimate goal the production of valid and trustworthy inferences."
Content analysis has come to be a sort of 'umbrella term' referring to an almost boundless set of quite diverse research approaches and techniques. Broadly, it can refer to methods for studying and/or retrieving meaningful information from documents.[1] In a more focused way, content analysis refers to a family of techniques for studying the "mute evidence" of texts and artifacts.[2] There are 5 types of texts in content analysis:
Content analysis can also be described as studying traces, which are documents from past times, and artifacts, which are non-linguistic documents. Texts are understood to b

We now have all the text from the page, split up by paragraph. If we wanted to get the section headers or references as well it would require a bit more work, but is doable.

There is one more thing we might want to do before sending this text to be processed, remove the references indicators (`[2]`, `[3]` , etc). To do this we can use a short regular expression.

In [8]:
contentParagraphs = []
for pTag in contentPTags:
    contentParagraphs.append(re.sub(r'\[\d+\]', '', pTag.text))

#convert to a DataFrame
contentParagraphsDF = pandas.DataFrame({'paragraph-text' : contentParagraphs})
print(contentParagraphsDF)

                                       paragraph-text
0   Content analysis is "a wide and heterogeneous ...
1   Content analysis has come to be a sort of 'umb...
2   Content analysis can also be described as stud...
3   Despite the wide variety of options, generally...
4   Over the years, content analysis has been appl...
5   In recent times, particularly with the advent ...
6   Quantitative content analysis has enjoyed a re...
7                                                    
8                                                    
9   The method of content analysis enables the res...
10  Since the 1980s, content analysis has become a...
11  The creation of coding frames is intrinsically...
12  Mimetic Convergence thus aims to show the proc...
13  Every content analysis should depart from a hy...
14  As an evaluation approach, content analysis is...
15  Qualitative content analysis is “a systematic,...
16  Holsti groups fifteen uses of content analysis...
17  He also places these use

Now we have a `DataFrame` of all the relevant text from the page ready to be processed

# Spidering

What if we want to to get a bunch of different pages from wikipedia. We would need to get the url of each of the pages we want, usually we will want pages that are linked to by other pages, so we will need to parse pages and find the links. Right now we will be getting all the links in the body of the content analysis page.

To do this we will need to find all the `<a>` (anchor) tags with `href`s inside of `<p>` tags.

In [9]:
tagLinks = []
for pTag in contentPTags:
    #we only want hrefs that link to wiki pages
    tagLinks += pTag.findAll('a', href=re.compile('/wiki/'), class_=False)
print(tagLinks[:4])

[<a href="/wiki/Text_(literary_theory)" title="Text (literary theory)">texts</a>, <a href="/wiki/Trace_evidence" title="Trace evidence">traces</a>, <a href="/wiki/Abductive_reasoning" title="Abductive reasoning">abduction</a>, <a href="/wiki/Hermeneutics" title="Hermeneutics">Hermeneutics</a>]


We will be adding these new texts to our DataFrame `contentParagraphsDF` so we will need to add 2 more columns to keep track of paragraph numbers and sources.

In [10]:
contentParagraphsDF['source'] = [wikipedia_content_analysis] * len(contentParagraphsDF['paragraph-text'])
contentParagraphsDF['paragraph-number'] = range(len(contentParagraphsDF['paragraph-text']))
contentParagraphsDF

,paragraph-text,source,paragraph-number
0,"Content analysis is ""a wide and heterogeneous ...",https://en.wikipedia.org/wiki/Content_analysis,0
1,Content analysis has come to be a sort of 'umb...,https://en.wikipedia.org/wiki/Content_analysis,1
2,Content analysis can also be described as stud...,https://en.wikipedia.org/wiki/Content_analysis,2
3,"Despite the wide variety of options, generally...",https://en.wikipedia.org/wiki/Content_analysis,3
4,"Over the years, content analysis has been appl...",https://en.wikipedia.org/wiki/Content_analysis,4
5,"In recent times, particularly with the advent ...",https://en.wikipedia.org/wiki/Content_analysis,5
6,Quantitative content analysis has enjoyed a re...,https://en.wikipedia.org/wiki/Content_analysis,6
7,,https://en.wikipedia.org/wiki/Content_analysis,7
8,,https://en.wikipedia.org/wiki/Content_analysis,8
9,The method of content analysis enables the res...,https://en.wikipedia.org/wiki/Content_analysis,9


Then we can define a function to parse each linked page and add its text to our DataFrame.

In [11]:
#wikipedia_base_url = 'https://en.wikipedia.org'

def getTextFromWikiPage(linkTag):
    #We need to extract the url from the <a> tag
    relurl = linkTag.get('href')
    #The urls are relative so we need to prepend the wikipedia url
    #while both of them are strings using the specialized function means
    #badly formatted relurls will be fixed, if possible
    url = urllib.parse.urljoin(wikipedia_base_url, relurl)
    #Make a dict to store data before adding it to the DataFrame
    parsDict = {'source' : [], 'paragraph-number' : [], 'paragraph-text' : []}
    #Now we get the page
    r = requests.get(url)
    soup = bs4.BeautifulSoup(r.text, 'html.parser')
    #enumerating gives use the paragraph number
    for parNum, pTag in enumerate(soup.body.findAll('p')):
        #same regex as before
        parsDict['paragraph-text'].append(re.sub(r'\[\d+\]', '', pTag.text))
        parsDict['paragraph-number'].append(parNum)
        parsDict['source'].append(url)
    return pandas.DataFrame(parsDict)

And run it on our list of link tags

In [12]:
for aTag in tagLinks[:5]:
    #ignore_index means the indices will not be reset after each append
    contentParagraphsDF = contentParagraphsDF.append(getTextFromWikiPage(aTag),ignore_index=True)
contentParagraphsDF

,paragraph-number,paragraph-text,source
0,0,"Content analysis is ""a wide and heterogeneous ...",https://en.wikipedia.org/wiki/Content_analysis
1,1,Content analysis has come to be a sort of 'umb...,https://en.wikipedia.org/wiki/Content_analysis
2,2,Content analysis can also be described as stud...,https://en.wikipedia.org/wiki/Content_analysis
3,3,"Despite the wide variety of options, generally...",https://en.wikipedia.org/wiki/Content_analysis
4,4,"Over the years, content analysis has been appl...",https://en.wikipedia.org/wiki/Content_analysis
5,5,"In recent times, particularly with the advent ...",https://en.wikipedia.org/wiki/Content_analysis
6,6,Quantitative content analysis has enjoyed a re...,https://en.wikipedia.org/wiki/Content_analysis
7,7,,https://en.wikipedia.org/wiki/Content_analysis
8,8,,https://en.wikipedia.org/wiki/Content_analysis
9,9,The method of content analysis enables the res...,https://en.wikipedia.org/wiki/Content_analysis
